# fairseq多进程机制

> 本期源码来自fairseq-0.2.0

fairseq中的多进程由三个部分组成，核心模块是**MultiprocessingEventLoop**，一个多进程的事件循环调度器；主要功能模块是**MultiprocessingTrainer**，Trainer模块是EventLoop的一个包装，负责实现训练的核心逻辑；还有一个辅助模块MultiprocessingPdb，该模块继承了pdb.Pdb，用于在程序运行中动态的对程序进行调试。

## MultiprocessingEventLoop

EventLoop对象在初始化时会做两件事，**_start_error_handler()**和**self._start_multiprocessing()**。

**self._start_error_handler()**会建立一个错误处理队列，并且在主进程中启动一个线程用来处理子进程中产生的所有异常。线程中会阻塞获取错误处理队列中的元素，当队列为空时，线程被阻塞；当队列中出现错误时，则主动结束进程。

**self._start_multiprocessing()**是实际上的工作函数。函数运行前，先根据新建多进程的方式，来获得当前进程环境/上下文(context)，使用变量self._mp来存放。在该函数的函数体中，首先会根据参数device_ids的数量来决定进程的数量。确定进程的数量后，针对每个进程会建立两组管道对象，来与进程进行通信，其中两个存放在主对象中，备子进程的输入输出使用，将剩余两个管道对象作为建立进程时的参数，供子进程内部输入输出使用。接着使用进程上下文对象self._mp新建进程。每个子进程的工作函数都是**self._process_event_loop**，工作函数中内嵌一个循环，每个循环将从输入中获取一个二元组，分别是函数名，与函数参数。有了函数名之后，子进程将以函数名为key，从**MultiprocessingEventLoop**获取对应的函数体，然后以传入的参数来执行对应的函数体。执行过程中遇到的未知异常将被放入错误处理队列，由主进程负责处理。

该类实现一个**self.call_async**供子类使用，该函数的功能是通过输入管道向子进程发送参数，使子进程执行，并通过输出管道异步返回结果。返回的结果由源码文件中的**Future**类进行包裹处理。

最后，本类还提供了**self.stop**函数供调用者显式停止各个子线程。

## MultiprocessingTrainer

**MultiprocessingTrainer**继承自**MultiprocessingEventLoop**，由父类提供多进程运行的必要部件，本类主要负责提供多进程执行中需要的各种业务函数。

**MultiprocessingTrainer**类下的函数众多，但是可以分为以下几类：
**_async_**开始的函数：
* _async_init
* _async_forward
* _async_backward_and_opt


**_build_**开始的函数：
* _build_optimizer
* _build_lr_scheduler

处理io的函数，该类函数多转到_async_[func_name]来完成功能，该类函数锁对应的async函数在上面不再重复列出
* get_model
* save_checkpoint
* load_checkpoint
* set_seed
* get_lr

训练函数：
* train_step
* valid_step
* lr_step 不清楚是否该放在这里

其他：
* _all_reduce_and_rescale_grads
* _scatter_samples -> _async_prepare_sample
* 

## MultiprocessingPdb

多进程工作三大巨头的最后一位是**MultiprocessingPdb**，这个类是最简单的一个类，其实现的目的就是使pdb在多进程的环境下可以使用。所以实现的重点就是加锁，和获得std_in的文件句柄。